In [ ]:
# Tensorflow to ONNX conversion is supported through the tf2onnx converter.

import tensorflow as tf
import tf2onnx

In [ ]:
# Change this path to the input name of your tf model
input_model_name = "test"

# Change this path to the output name and path for the ONNX model
output_onnx_model = "model.onnx"

If you already have a frozen TF graph, you can skip this step and simply import your model.

In [ ]:
with tf.Session() as sess:
    x = tf.placeholder(tf.float32, [2, 3], name="input")
    x_ = tf.add(x, x)
    _ = tf.identity(x_, name="output")
    onnx_graph = tf2onnx.tfonnx.process_tf_graph(sess.graph, input_names=["input:0"], output_names=["output:0"])
    onnx_model = onnx_graph.make_model(input_model_name)

In [ ]:
# Save as protobuf

onnxmltools.utils.save_model(onnx_model, output_onnx_model)